# SETUP

In [ ]:
!git clone https://github.com/rmihaylov/falcontune.git
!wget https://huggingface.co/TheBloke/falcon-40b-instruct-GPTQ/resolve/main/gptq_model-4bit--1g.safetensors

In [ ]:
# Installation:
!cd falcontune && pip install -r requirements.txt
!cd falcontune && python setup.py install
# !cd falcontune && python setup_cuda.py install  # if cuda, default is triton

In [ ]:
# Restart:
import os; os.kill(os.getpid(), 9)

# GENERATE

In [ ]:
!falcontune generate \
    --interactive \
    --model falcon-40b-instruct-4bit \
    --weights gptq_model-4bit--1g.safetensors \
    --max_new_tokens 50 \
    --use_cache \
    --do_sample \
    --prompt "How to prepare pasta?" \
    --backend triton

# FINETUNE

In [ ]:
!wget https://github.com/gururise/AlpacaDataCleaned/raw/main/alpaca_data_cleaned.json

In [ ]:
# Disable wandb:
import os; os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!falcontune finetune \
    --model=falcon-40b-instruct-4bit \
    --weights=./gptq_model-4bit--1g.safetensors \
    --dataset=./alpaca_data_cleaned.json \
    --data_type=alpaca \
    --lora_out_dir=./falcon-40b-instruct-4bit-alpaca/ \
    --mbatch_size=1 \
    --batch_size=2 \
    --epochs=3 \
    --lr=3e-4 \
    --cutoff_len=256 \
    --lora_r=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \
    --save_steps=50 \
    --save_total_limit=3 \
    --logging_steps=5 \
    --target_modules='["query_key_value"]' \
    --backend=triton

# LOAD

In [ ]:
!falcontune generate \
    --interactive \
    --model falcon-40b-instruct-4bit \
    --weights ./gptq_model-4bit--1g.safetensors \
    --lora_apply_dir falcon-40b-instruct-4bit-alpaca/ \
    --max_new_tokens 50 \
    --use_cache \
    --do_sample \
    --instruction "How to prepare pasta?" \
    --backend triton